with sge

In [1]:
from ecell4 import *
%matplotlib inline
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
from ecell4.extra import ensemble

def wrap(job,job_id,task_id):
    def singlerun(kaf,run,phic):
        L = 2.
        rm=0.005
        duration=int(L/rm/4)*4#*2
        voxel_radius = rm*1.021
        D = 1.
        kD = 4 * math.pi * (2 * rm) * D
        ka = kD * kaf
        dt = (4 * voxel_radius * voxel_radius) / (6 * D)    
        maxt=dt*duration
        rng = GSLRandomNumberGenerator()  
        rng.seed(run)
        Nc=int(phic*L**3/(4*math.pi*rm**3 /3))
        with species_attributes():
            C | {'D':str(D),'radius':str(voxel_radius)}
            B | {'D':str(0),'radius':str(voxel_radius)}
            T | {'D':str(D),'radius':str(voxel_radius)}
        with reaction_rules():
            T + B >B | ka     
        m=get_model()           
        w = spatiocyte.SpatiocyteWorld(ones() * L, voxel_radius, rng)
        w.bind_to(m)       
        coord1 = w.position2coordinate(ones() * L * 0.5)
        coord2 = w.get_neighbor(coord1, 0)
        (pidB,p),suc=w.new_particle(Species('B'),w.coordinate2position(coord1))  
        (pidT,p),suc=w.new_particle(Species('T'),w.coordinate2position(coord2))     
        w.add_molecules(Species('C'),Nc)
        t= 0.0
        sim = spatiocyte.SpatiocyteSimulator(w)
        sim.initialize()
        pos = tuple(p.position())
        while all(0<= dim <L for dim in pos) and t<maxt:
            NT=w.num_particles(Species('T'))        
            if NT==0:
                return True
            else:
                (pid,p)=w.get_particle(pidT)
                pos = tuple(p.position())
                sim.step()
                t = sim.t()
        return False    
    job.update({'run':task_id})
    out=singlerun(**job)
    return out

simrun=2000
nproc=300
phic=0.3
kafs=[1.]
result=[('phic','kaf','pt','psim')]
for kaf in kafs:
    jobs=[{'kaf':kaf,'phic':phic}]
    #res=ensemble.run_sge(wrap,jobs,n=simrun,nproc=nproc,path='tmp',modules=('numpy','ecell4','math'))[0]
    pt = 1./(1+1./kaf)
    print('kaf={},phic={},Prebtotal_theory{:.3}'.format(kaf,phic,pt))
    psim = float(res.count(True))/len(res)
    print('preb_sim',psim)  
    result.append((phic,kaf,pt,psim))


kaf=1.0,phic=0.3,Prebtotal_theory0.5


NameError: name 'res' is not defined

In [14]:
result

[('phic', 'kaf', 'pt', 'psim'), (0.3, 1.0, 0.5, 0.6365)]

In [4]:
L=1
[('phic', 'kaf', 'pt', 'psim'),
 (0.001, 0.01, 0.009900990099009901, 0.01),
 (0.001, 1.0, 0.5, 0.4638),
 (0.001, 100, 0.9900990099009901, 0.9914)]
L=4.
[('phic', 'kaf', 'pt', 'psim'),
 (0.001, 0.01, 0.009900990099009901, 0.0122),
 (0.001, 1.0, 0.5, 0.4966),
 (0.001, 100, 0.9900990099009901, 0.9896)]
L=1.
[('phic', 'kaf', 'pt', 'psim'),
 (0.3, 0.01, 0.009900990099009901, 0.0126),
 (0.3, 1.0, 0.5, 0.592),
 (0.3, 100, 0.9900990099009901, 0.9952)]
L=1.5, simrun=1000,
(0.3,1,0.5,0.594)
L=1, simrun=2000
[('phic', 'kaf', 'pt', 'psim'),
 (0.3, 1.0, 0.5, 0.6025),
 (0.3, 5.0, 0.8333333333333334, 0.8755),
 (0.3, 10.0, 0.9090909090909091, 0.9305)]
L=1, dur=4x
[('phic', 'kaf', 'pt', 'psim'), 
 (0.3, 1.0, 0.5, 0.6155)]
L=1,dur=10x
[('phic', 'kaf', 'pt', 'psim'), 
 (0.3, 1.0, 0.5, 0.623)]
L=1,dur=100x
[('phic', 'kaf', 'pt', 'psim'), 
 (0.3, 1.0, 0.5, 0.6365)]
L=1,dur=100x
0.3,0.01,0.0099,0.0165
0.3,100,0.99,0.997

SyntaxError: invalid syntax (<ipython-input-4-1bb406b71b34>, line 23)

In [8]:
kf=[1,5,10]
p=[0.603,0.876,0.931]
for i,j in zip(kf,p):
    print(0.42/(1./j -1)/i)

0.6379345088161209
0.5934193548387094
0.5666956521739127


In [6]:
[0.42/(1./p-1)/k for p,k in [(0.592,1.),(0.0126,0.01),(0.9952,100)]]

[0.6094117647058823, 0.535953007899534, 0.8707999999999886]

with multiprocessing

In [2]:
from ecell4 import *
%matplotlib inline
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
from ecell4.extra import ensemble

def wrap(job,job_id,task_id):
    def singlerun(kaf,run,phic):
        L = 1.
        rm=0.005
        duration=int(L/rm/4)*100
        voxel_radius = rm*1.021
        D = 1.
        kD = 4 * math.pi * (2 * rm) * D
        ka = kD * kaf
        dt = (4 * voxel_radius * voxel_radius) / (6 * D)    
        maxt=dt*duration
        rng = GSLRandomNumberGenerator()  
        rng.seed(run)
        Nc=int(phic*L**3/(4*math.pi*rm**3 /3))
        with species_attributes():
            C | {'D':str(D),'radius':str(voxel_radius)}
            B | {'D':str(0),'radius':str(voxel_radius)}
            T | {'D':str(D),'radius':str(voxel_radius)}
        with reaction_rules():
            T + B >B | ka     
        m=get_model()           
        w = spatiocyte.SpatiocyteWorld(ones() * L, voxel_radius, rng)
        w.bind_to(m)       
        coord1 = w.position2coordinate(ones() * L * 0.5)
        coord2 = w.get_neighbor(coord1, 0)
        (pidB,p),suc=w.new_particle(Species('B'),w.coordinate2position(coord1))  
        (pidT,p),suc=w.new_particle(Species('T'),w.coordinate2position(coord2))     
        w.add_molecules(Species('C'),Nc)
        t= 0.0
        sim = spatiocyte.SpatiocyteSimulator(w)
        sim.initialize()
        pos = tuple(p.position())
        while all(0<= dim <L for dim in pos) and t<maxt:
            NT=w.num_particles(Species('T'))        
            if NT==0:
                return True
            else:
                (pid,p)=w.get_particle(pidT)
                pos = tuple(p.position())
                sim.step()
                t = sim.t()
        return False    
    job.update({'run':task_id})
    out=singlerun(**job)
    return out

simrun=2000
nproc=70
phic=0.3
kafs=[0.01,100]
result=[('phic','kaf','pt','psim')]
for kaf in kafs:
    jobs=[{'kaf':kaf,'phic':phic}]
    res=ensemble.run_multiprocessing(wrap,jobs,n=simrun,nproc=nproc,modules=('numpy','ecell4','math'))[0]
    #res=ensemble.run_sge(wrap,jobs,n=simrun,nproc=nproc,path='tmp',modules=('numpy','ecell4','math'))[0]
    pt = 1./(1+1./kaf)
    print('kaf={},phic={},Prebtotal_theory{:.3}'.format(kaf,phic,pt))
    psim = float(res.count(True))/len(res)
    print('preb_sim',psim)  
    result.append((phic,kaf,pt,psim))


kaf=0.01,phic=0.3,Prebtotal_theory0.0099
preb_sim 0.0165
kaf=100,phic=0.3,Prebtotal_theory0.99
preb_sim 0.997


In [3]:
1./(1+1./100)

0.9900990099009901

In [ ]:
L,t,p
1,1e5,0.4
1,1e6,0.4
2,1e5,0.46
2,1e4,0.46
2,1e2,0.44
4,1e2,0.5

In [ ]:
from ecell4 import *
import numpy as np
%matplotlib inline
import math
import sys
import matplotlib.pyplot as plt
from ecell4.extra import ensemble
import time

def singlerun(kaf,run,phic):
    rm=0.005
    L = 1.   
    duration=1e3
    voxel_radius = rm*1.021
    D = 1.
    kD = 4 * math.pi * (2 * rm) * D
    ka = kD * kaf
    Pacc = ka / (6 * math.sqrt(2) * D * voxel_radius)
    if (Pacc>1):
        alpha=1./Pacc
    else: alpha=1.
    assert Pacc*alpha <=1
    assert alpha  <= 1    
    dt = (4 * voxel_radius * voxel_radius) / (6 * D)    
    maxt=dt*duration
    rng = GSLRandomNumberGenerator()   
    rng.seed(run)
    Nc=int(phic*L**3/(4*math.pi*rm**3 /3))
    
    with species_attributes():
        C | {'D':str(D),'radius':str(voxel_radius)}
        T | {'D':str(0),'radius':str(voxel_radius)}
    m=get_model()           
    w = spatiocyte.create_spatiocyte_world_cell_list_impl(ones() * L, voxel_radius, Integer3(3, 3, 3), rng)
    w.bind_to(m)        
    w.add_molecules(Species('C'),Nc)

    coord1 = w.position2coordinate(ones() * L * 0.5)
    coord2 = w.get_neighbor(coord1, 0)
    (pidT1,p),suc=w.new_particle(Species('T'),w.coordinate2position(coord2))   
    dt = dt*alpha
    t= 0.0
    sim = spatiocyte.SpatiocyteSimulator(w)
    sim.initialize()
    sim.set_dt(dt)
    
    while all(0<= dim <L for dim in w.coordinate2position(coord2)) and t<maxt:
        rnd = w.rng().uniform_int(0, 11)
        newcoord = w.get_neighbor(coord2, rnd)          
        if newcoord == coord1:
            if Pacc*alpha >= w.rng().uniform(0, 1):
                return True
        elif alpha>=w.rng().uniform(0,1):
            (pidT2,p),suc=w.new_particle(Species('T'),w.coordinate2position(newcoord))
            if suc:
                coord2 = newcoord
                w.remove_particle(pidT1)   
                pidT1=pidT2
        sim.step()
        t = sim.t()
    assert(w.num_particles(Species('T'))==1)
    return False

def wrap(job,job_id,task_id):
    job.update({'run':task_id})
    out=singlerun(**job)
    return out

simrun=100
nproc=70
phic=0.01
kafs=[1.]
st=time.time()
for kaf in kafs:
    jobs=[{'kaf':kaf,'phic':phic}]
    res=ensemble.run_multiprocessing(wrap,jobs,n=simrun,nproc=nproc)[0]
    print('kaf={},phic={},Prebtotal_theory{:.3}'.format(kaf,phic,1./(1+1./kaf)))
    print('preb_sim',float(res.count(True))/len(res))    
print(time.time()-st)
#name='/home/chew/outputdata/LS3Dcrowd_Dt{}Dc{}phi{:.3}phit{:.3}Nt{}rm{:.3}L{}'.format(Dt,Dc,phi,phit,Nt,rv,L)
#f=open(name,'w')
#np.savetxt(name,np.column_stack([tt,msd]),delimiter=',')
#f.close()    

In [7]:
r=0.0025
D=[0.06,4]
kd=4*3.14*r*2*D[1]
ka=[0.04,0.1]
[k/kd for k in ka]

[0.1592356687898089, 0.39808917197452226]